First, we install pandas

In [50]:
!pip install pandas

Then we import it.

In [51]:
import pandas as pd

We read all the data from the csv to a DataFrame in order to manipulate it properly.

In [52]:
url = 'product_data.csv'
rawdata = pd.read_csv(url, index_col=0)

We visualize it's shape and head to see how is it.

In [53]:
print(rawdata.shape)
print(rawdata.head)

(9222, 12)
<bound method NDFrame.head of                  cod_color_code des_color_specification_esp  \
cod_modelo_color                                              
41085800-02                  02                    OFFWHITE   
53000586-TO                  TO               TEJANO OSCURO   
53030601-81                  81                 ROSA PASTEL   
53050730-15                  15                     MOSTAZA   
53070773-70                  70                        ROJO   
...                         ...                         ...   
57066027-99                  99                       NEGRO   
57076028-92                  92                        GRIS   
57068275-05                  05                       CRUDO   
57067125-78                  78                     BURDEOS   
57039120-PL                  PL                       PLATA   

                 des_agrup_color_eng des_sex des_age des_line      des_fabric  \
cod_modelo_color                                          

In particular we want to see which colums we have.

In [54]:
print(rawdata.columns)

Index(['cod_color_code', 'des_color_specification_esp', 'des_agrup_color_eng',
       'des_sex', 'des_age', 'des_line', 'des_fabric', 'des_product_category',
       'des_product_aggregated_family', 'des_product_family',
       'des_product_type', 'des_filename'],
      dtype='object')


From the dataset description:

1. cod_modelo_color - A unique identifier for a particular fashion product based on its model and color.
2. cod_color_code - A code indicating the specific color of the product.
3. des_color_specification_esp - Description of the color in Spanish.
4. des_agrup_color_eng - Grouped color description in English.
5. des_sex - Gender for which the product is intended (e.g., "Unisex", "Female", "Male").
6. des_age - Age group for which the product is intended (e.g., "Adult", "Child").
7. des_line - The line or collection to which the product belongs.
8. des_fabric - The fabric or material of the product.
9. des_product_category - Broad category of the product (e.g., "Bottoms", "Tops").
10. des_product_aggregated_family - An aggregated family description of the product (e.g., "Trousers & leggings").
11. des_product_family - Family description of the product (e.g., "Trousers", "Shirts").
12. des_product_type - Type of the product within its family (e.g., "Trousers", "Shirt").
13. des_filename - File path of the product image associated with the cod_modelo_color. Images can be found in the images folder. Image is always the frontal image of the product without a human model.


All the data is categorical, so we should be careful and preprocess it.
We will go column by column and try to reduce the number of unique values by removing outliers

First we'll process `cod_counts`.

In [55]:
cod_counts = rawdata['cod_color_code'].value_counts()
print(cod_counts)

99    2119
05     608
02     563
OR     445
01     426
      ... 
TL       1
66       1
93       1
73       1
97       1
Name: cod_color_code, Length: 110, dtype: int64


We see that there are a lot of unique values, but most of them only appear once, let's remove the rows with those rare values.

In [56]:
MIN_COD_FREQ = 30
ndata1 = rawdata[rawdata['cod_color_code'].isin(cod_counts[cod_counts.gt(MIN_COD_FREQ)].index)]

In [57]:
print(ndata1['cod_color_code'].value_counts().shape)
print(rawdata.shape)
print(ndata1.shape)

(48,)
(9222, 12)
(8606, 12)


We have removed about 600 rows, but now there are less colors.

In [58]:
agrup_counts = ndata1['des_agrup_color_eng'].value_counts()
print(agrup_counts)

GREY      2824
WHITE     2769
BLUE       849
BROWN      655
RED        348
GREEN      347
PINK       346
YELLOW     218
ORANGE     169
PURPLE      81
Name: des_agrup_color_eng, dtype: int64


We can see `des_agrup_color_eng` does not need any work.

Now we are going to drop all rows except ones that have products for Female Adults from the line SHE, since that's what happens the majority of the times and since that allows us to remove the colums that contain that data.

In [59]:
ndata2 = ndata1[(ndata1['des_sex'] == 'Female') & (ndata1['des_age'] == 'Adult') & (ndata1['des_line'] == 'SHE')]
print(ndata2.shape)
print(ndata2.head)

(8532, 12)
<bound method NDFrame.head of                  cod_color_code des_color_specification_esp  \
cod_modelo_color                                              
41085800-02                  02                    OFFWHITE   
53000586-TO                  TO               TEJANO OSCURO   
53030601-81                  81                 ROSA PASTEL   
53050730-15                  15                     MOSTAZA   
53070773-70                  70                        ROJO   
...                         ...                         ...   
57066027-99                  99                       NEGRO   
57076028-92                  92                        GRIS   
57068275-05                  05                       CRUDO   
57067125-78                  78                     BURDEOS   
57039120-PL                  PL                       PLATA   

                 des_agrup_color_eng des_sex des_age des_line      des_fabric  \
cod_modelo_color                                          

In [60]:
ndata3 = ndata2.drop(['des_sex', 'des_age', 'des_line'], axis=1)
print(ndata3.shape)
print(ndata3.head)

(8532, 9)
<bound method NDFrame.head of                  cod_color_code des_color_specification_esp  \
cod_modelo_color                                              
41085800-02                  02                    OFFWHITE   
53000586-TO                  TO               TEJANO OSCURO   
53030601-81                  81                 ROSA PASTEL   
53050730-15                  15                     MOSTAZA   
53070773-70                  70                        ROJO   
...                         ...                         ...   
57066027-99                  99                       NEGRO   
57076028-92                  92                        GRIS   
57068275-05                  05                       CRUDO   
57067125-78                  78                     BURDEOS   
57039120-PL                  PL                       PLATA   

                 des_agrup_color_eng      des_fabric  \
cod_modelo_color                                       
41085800-02                 

Now we can drop some of the uncommon `des_product_family` values, some of which may be more difficult to create combinations with, like fragances and swimwear. We will remove those as well af some other outliers.

In [61]:
fam_counts = ndata3['des_product_family'].value_counts()
print(fam_counts)

Dresses                     1106
Shirt                        634
Sweater                      607
T-shirt                      598
Trousers                     572
Jewellery                    570
Bags                         560
Tops                         503
Skirts                       452
Footwear                     444
Jeans                        286
Hats, scarves and gloves     253
Jackets                      193
Jumpsuit                     170
Wallets & cases              168
Coats                        164
Belts and Ties               156
Cardigans                    156
Shorts                       144
Blazers                      141
Intimate                     129
Glasses                      124
Puffer coats                  64
Sweatshirts                   54
Swimwear                      44
Vest                          42
Trenchcoats                   42
Leggings and joggers          39
Fragances                     30
Bodysuits                     26
Gadgets   

In [62]:
MIN_FAM_FREQ = 45
ndata4 = ndata3[ndata3['des_product_family'].isin(fam_counts[fam_counts.gt(MIN_FAM_FREQ)].index)]

In [63]:
print(ndata4['des_product_family'].value_counts().shape)
print(ndata3.shape)
print(ndata4.shape)

(24,)
(8532, 9)
(8248, 9)


Now we will drop some other columns that are either too specific or contain almost identical data to another column.

In [64]:
todrop = ['des_color_specification_esp','des_product_aggregated_family','des_product_type']
ndata5 = ndata4.drop(todrop, axis=1)

In [65]:
defdata = ndata5

We can overview the remaining data. We have dropped less than 10% of the dataset and have 5 remaining columns.

In [66]:
print(defdata.shape)
print(defdata.head)

(8248, 6)
<bound method NDFrame.head of                  cod_color_code des_agrup_color_eng      des_fabric  \
cod_modelo_color                                                      
41085800-02                  02               WHITE         P-PLANA   
53000586-TO                  TO                BLUE         J-JEANS   
53030601-81                  81                PINK         P-PLANA   
53050730-15                  15              YELLOW         P-PLANA   
53070773-70                  70                 RED         P-PLANA   
...                         ...                 ...             ...   
57066027-99                  99                GREY         P-PLANA   
57076028-92                  92                GREY         P-PLANA   
57068275-05                  05               WHITE  C-COMPLEMENTOS   
57067125-78                  78                 RED      K-CIRCULAR   
57039120-PL                  PL                GREY        T-TRICOT   

                                 des

In [67]:
for column in defdata:
  print(column, defdata[column].value_counts().shape)

cod_color_code (48,)
des_agrup_color_eng (10,)
des_fabric (7,)
des_product_category (6,)
des_product_family (24,)
des_filename (8248,)


As we can see above the number of unique values in each column and the number of columns has been reduced. This is very useful for AI models that can use One Hot Encoding more easily or for any other application.

In [68]:
defdata.to_csv('clean_product_data_with_filenames.csv')

In the end we save our obtained data.

In [69]:
defdatanofile = defdata.drop('des_filename', axis=1)
defdatanofile.to_csv('clean_product_data.csv')

We may also want a version without files for other applications.